<a href="https://colab.research.google.com/github/Mathufnn/congresso_artificial/blob/master/interpretador_de_leis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# autentica o uso do drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# descomprime as pastas de arquivos na cache do notebook. nao afeta o google drive

!unzip /content/drive/My\ Drive/congresso_artificial/dataset_treinamento_arquivo.zip
!unzip /content/drive/My\ Drive/congresso_artificial/lista_votacoes_xml.zip

Archive:  /content/drive/My Drive/congresso_artificial/dataset_treinamento_arquivo.zip
replace dataset_treinamento/materia10_pag10_ano2010.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/drive/My Drive/congresso_artificial/lista_votacoes_xml.zip
replace lista_votacoes_xml/lista_votacoes1991.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
# verifica se está usando a GPU 
# saida esperada: '/device:GPU:0'

import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import text_to_word_sequence
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

####

directory = '/content/dataset_treinamento/'
raw_text = ""

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
      with open(directory + filename, 'r') as f:
        raw_text += f.read() + " "

####

words = text_to_word_sequence(raw_text, filters='"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n', split=' ', lower=True)

MAX_WORDS = 100000
words = words[0:MAX_WORDS]

dict_words = sorted(set(words))

n_words = len(words)
n_vocab = len(dict_words)
print("Número de palavras: ", n_words)
print("Tamanho do vocabulário: ", n_vocab)

Número de palavras:  100000
Tamanho do vocabulário:  9955


In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import text_to_word_sequence
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

# filename = "dom_casmurro.txt"
# raw_text = open(filename).read()
# raw_text = raw_text.lower()

# punch = ['.', '[', ']', '(', ')', ';', ':', "'", '/', '"', ',', '?', '*', '!', '-', '$', '%', '&', '\n']

# for i in punch:    
#     raw_text = raw_text.replace(i, ' ' + i + ' ')

# words = text_to_word_sequence(raw_text, filters='"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n', split=' ', lower=True)
# dict_words = sorted(set(words))

# print("Número de palavras: ", n_words)
# print("Tamanho do vocabulário: ", n_vocab)

seq_length = 50
dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
  seq_in = words[i:i + seq_length]
  seq_out = words[i + seq_length]
  dataX.append([word for word in seq_in])
  dataY.append(seq_out)
n_patterns = len(dataX)
print("Número de Instâncias: ", n_patterns)

print("haha1") # metodo avancado de debugging

tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(dataX)
X = tokenizer.texts_to_sequences(dataX)
y = tokenizer.texts_to_sequences(dataY)

print("haha2")

flat_y = [item for sublist in y for item in sublist]

print('Numero x: ', len(X))
print('Numero y: ', len(flat_y))

print("haha3")

X = numpy.reshape(X, (n_patterns, seq_length))
y = np_utils.to_categorical(flat_y)

print("haha4")

hidden_size = 128
dropout=0.6
model = Sequential()
model.add(Embedding(n_vocab, 1024, input_length=seq_length))
model.add(GRU(hidden_size, return_sequences=True, dropout=dropout))
model.add(GRU(hidden_size, dropout=dropout))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

print("haha5")

filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=25, batch_size=256, callbacks=callbacks_list)

Número de Instâncias:  99950
haha1
haha2
Numero x:  99950
Numero y:  99950
haha3
haha4




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


haha5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/25





99950/99950 [==============================] - 92s 922us/step - loss: 6.9517

Epoch 00001: loss improved from inf to 6.95168, saving model to weights-improvement-01-6.9517.hdf5
Epoch 2/25
99950/99950 [==============================] - 82s 820us/step - loss: 6.2506

Epoch 00002: loss improved from 6.95168 to 6.25060, saving model to weights-improvement-02-6.2506.hdf5
Epoch 3/25
99950/99950 [==============================] - 83s 827us/step - loss: 5.7576

Epoch 00003: loss improved from 6.25060 to 5.75758, saving model to weights-improvement-03-5.7576.hdf5
Epoch 4/25
99950/99950 [==============================] - 82s 825us/step - loss: 5.3953

Epoch 00004: loss impro

In [0]:
print(X)
print(y.shape[1])

[919, 1308, 442, 3, 1646, 1487, 422, 7, 328, 34, 10, 1007, 1008, 9591, 7, 73, 4, 3328, 6094, 4, 7, 168, 411, 10, 3, 886, 5, 1443, 20, 166, 6, 1103, 2, 20, 14, 7, 20, 14, 7, 632, 146, 643, 9, 10118, 1188, 9, 194, 1, 371, 1]
10241


In [0]:
import sys
import numpy

#seleciona um trecho aleatório do livro para servir como entrada
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
X = tokenizer.texts_to_sequences([pattern])[0]
print('Texto fornecido como entrada: ')
print("\"", ' '.join([value for value in pattern]), "\"")
print('Continuação do texto gerado pela rede: ')

probability = 1.0

for i in range(50):
  x_in = numpy.reshape(X, (1, seq_length))
  prediction = model.predict(x_in, verbose=0)
  index = numpy.argmax(prediction)
  probability *= prediction[0][index]

  result = tokenizer.sequences_to_texts([[index]])

  sys.stdout.write(result[0])
  sys.stdout.write(' ')
  X.append(index)
  X = X[1:len(X)]
  
print()
print('Probabilidade do texto gerado: ', probability)

Texto fornecido como entrada: 
" comissao de agricultura e reforma agraria cra de audiencia publica para discutir programa de producao de etanol social da amazonia para tanto peco o convite d o dr marcio silveira reitor da universidade federal do tocantins e djalma bezerra mello titular da superintendencia do desenvolvimento da amazonia sudam justificacao por "
Continuação do texto gerado pela rede: 
diversas razoes relacionadas a questao cunha assim o plano nacional para banda larga intitulado o brasil e reportagem daeletronorte e o processo internacional para a mulher da tv senado publicado no diario do senado federal o senado federal o protocolo sera assinado em bonfim rr em 14 de junho de 
Probabilidade do texto gerado:  8.2044921546984e-15


In [0]:
# importando os dois outros conjuntos de dados

import Pandas as pd


